# USGS Earthquake Data with pystac-monty

This notebook demonstrates how to use pystac-monty to process USGS earthquake data and visualize it using interactive maps. We'll:

1. Fetch the latest major earthquakes from USGS
2. Convert them to STAC items using pystac-monty
3. Display events on an interactive map
4. Allow selection of events to view related hazards and impacts
5. Explore the Monty STAC model and its metadata

Let's begin by importing the necessary libraries.

In [ ]:
# Basic libraries
import json
from datetime import datetime, timedelta

# Visualization libraries
import folium
import ipywidgets as widgets
import pandas as pd

# STAC and pystac-monty
import pytz
import requests
from folium.plugins import MarkerCluster
from IPython.display import clear_output, display

from pystac_monty.extension import MontyExtension
from pystac_monty.geocoding import WorldAdministrativeBoundariesGeocoder
from pystac_monty.sources.usgs import USGSDataSource, USGSTransformer

## 1. Fetch Recent Major Earthquakes from USGS

Let's fetch earthquakes with magnitude 5.5+ from the past 10 days using the USGS API.

In [ ]:
# Define USGS API endpoint for fetching earthquake data
def fetch_usgs_earthquakes(min_magnitude=5.5, days=60):
    """
    Fetch earthquake data from USGS API

    Parameters:
    - min_magnitude: Minimum magnitude to filter earthquakes (default: 4.5)
    - days: Number of days to look back (default: 30)

    Returns:
    - List of earthquake data as dictionaries
    """
    # Calculate the start time (days ago from now)
    start_time = datetime.now() - timedelta(days=days)
    start_time_str = start_time.strftime("%Y-%m-%dT%H:%M:%S")

    # USGS earthquake API endpoint
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

    # Parameters for the API request
    params = {"format": "geojson", "starttime": start_time_str, "minmagnitude": min_magnitude, "orderby": "time"}

    # Make the request to the USGS API
    response = requests.get(url, params=params)
    data = response.json()

    print(f"Found {len(data['features'])} earthquakes with magnitude {min_magnitude}+ in the last {days} days")
    return data


# Fetch recent major earthquakes
earthquake_data = fetch_usgs_earthquakes(min_magnitude=5.5, days=60)
earthquake_features = earthquake_data["features"]

In [ ]:
# Extract relevant information into a DataFrame
earthquakes_df = pd.DataFrame(
    [
        {
            "id": eq["id"],
            "title": eq["properties"]["title"],
            "time": datetime.fromtimestamp(eq["properties"]["time"] / 1000, pytz.UTC),
            "magnitude": eq["properties"]["mag"],
            "place": eq["properties"]["place"],
            "longitude": eq["geometry"]["coordinates"][0],
            "latitude": eq["geometry"]["coordinates"][1],
            "depth": eq["geometry"]["coordinates"][2],
            "tsunami": bool(eq["properties"].get("tsunami")),
            "felt": eq["properties"].get("felt") or 0,
        }
        for eq in earthquake_features
    ]
)

# Sort by magnitude (descending)
earthquakes_df = earthquakes_df.sort_values(by="magnitude", ascending=False)

# Display the DataFrame
earthquakes_df

## 2. Creating Synthetic STAC Items

Let's create synthetic STAC items from the earthquake data using the pystac-monty extension.

In [ ]:
# Initialize the geocoder
geocoder = WorldAdministrativeBoundariesGeocoder("../tests/data-files/world-administrative-boundaries.fgb")

# Create a list to store all synthetic STAC items
all_stac_items = []

# Loop through the earthquake data and create synthetic STAC items
for earthquake in earthquakes_df.to_dict(orient="records"):
    # Define the USGS API endpoint for fetching event detailed geojson
    event_url = f"https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/{earthquake['id']}.geojson"

    # Get the event data
    event_data = requests.get(event_url).text
    event_data_json = json.loads(event_data)

    # Find the losses data in the event data
    try:
        losses_data_url = event_data_json["properties"]["products"]["losspager"][0]["contents"]["json/losses.json"]["url"]
        losses_data = requests.get(losses_data_url).text
    except KeyError:
        losses_data = None

    # Setup the transformer
    data_source = USGSDataSource(event_url, event_data, losses_data)
    transformer = USGSTransformer(data_source, geocoder)

    # Create the synthetic STAC item
    items = transformer.make_items()
    all_stac_items.extend(items)

# Create synthetic STAC items
print(f"Created {len(all_stac_items)} STAC items from {len(earthquake_features)} earthquakes")

In [ ]:
# Separate the STAC items by role
event_items = []
hazard_items = []
impact_items = []

for item in all_stac_items:
    roles = item.properties.get("roles", [])
    if "event" in roles:
        event_items.append(item)
    elif "hazard" in roles:
        hazard_items.append(item)
    elif "impact" in roles:
        impact_items.append(item)

print(f"Events: {len(event_items)}, Hazards: {len(hazard_items)}, Impacts: {len(impact_items)}")

## 3. Displaying Earthquakes on a Map

Now, let's create an interactive map to display the earthquakes.

In [ ]:
# Function to create a map of earthquakes
def create_earthquake_map(event_items):
    # Calculate the average latitude and longitude for map centering
    if not event_items:
        return folium.Map(location=[0, 0], zoom_start=2)

    lats = [item.bbox[1] for item in event_items]
    lons = [item.bbox[0] for item in event_items]
    center_lat = sum(lats) / len(lats)
    center_lon = sum(lons) / len(lons)

    # Create the map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=2, tiles="CartoDB positron")

    # Add a title
    title_html = """
    <h3 align="center" style="font-size:20px">
        <b>Recent Major Earthquakes (M6.0+)</b>
    </h3>
    """
    m.get_root().html.add_child(folium.Element(title_html))

    # Create a marker cluster group
    marker_cluster = MarkerCluster(name="Earthquakes").add_to(m)

    # Add markers for each earthquake
    for item in event_items:
        # Get the earthquake properties
        mag = item.properties.get("eq:magnitude")
        title = item.properties.get("title")
        date_time = item.datetime.strftime("%Y-%m-%d %H:%M:%S UTC")

        # Calculate marker size and color based on magnitude
        size = max(6, mag * 3)  # Scale marker size

        # Color based on magnitude
        if mag >= 8.0:
            color = "darkred"
        elif mag >= 7.0:
            color = "red"
        elif mag >= 6.5:
            color = "orange"
        else:
            color = "yellow"

        # Create popup content
        popup_content = f"""
        <b>{title}</b><br>
        <b>Magnitude:</b> {mag}<br>
        <b>Time:</b> {date_time}<br>
        <b>ID:</b> {item.id}<br>
        <b>Depth:</b> {item.properties.get('eq:depth')} km<br>
        """

        # Add marker to the cluster
        folium.CircleMarker(
            location=[item.bbox[1], item.bbox[0]],
            radius=size,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=f"M{mag} - {title}",
        ).add_to(marker_cluster)

    # Add layer control
    folium.LayerControl().add_to(m)

    return m


# Create the earthquake map
earthquake_map = create_earthquake_map(event_items)
earthquake_map

## 4. Selecting Events to View Hazards and Impacts

Now, let's create a simple interface to select an earthquake event and view its associated hazards and impacts on a separate map.

In [ ]:
# Create a dropdown widget for selecting earthquakes
event_options = [(f"{item.properties.get('title')} ({item.id})", item.id) for item in event_items]
event_dropdown = widgets.Dropdown(
    options=event_options, description="Select Event:", style={"description_width": "initial"}, layout=widgets.Layout(width="80%")
)

# Create an output widget for displaying the map
map_output = widgets.Output()


# Function to find hazard and impact items for a given event ID
def find_related_items(event_id):
    # Extract the USGS ID from the event ID (format: usgs-event-{usgs_id})
    usgs_id = event_id.replace("usgs-event-", "")

    # Find related hazard items
    related_hazards = [item for item in hazard_items if usgs_id in item.id]

    # Find related impact items
    related_impacts = [item for item in impact_items if usgs_id in item.id]

    return related_hazards, related_impacts


# Function to create a map showing hazards and impacts for a selected event
def create_detail_map(event_id):
    # Find the selected event
    selected_event = next((item for item in event_items if item.id == event_id), None)

    if not selected_event:
        return folium.Map(location=[0, 0], zoom_start=2)

    # Find related hazards and impacts
    related_hazards, related_impacts = find_related_items(event_id)

    # Get event coordinates
    event_lat = selected_event.bbox[1]
    event_lon = selected_event.bbox[0]

    # Create the map centered on the event
    m = folium.Map(location=[event_lat, event_lon], zoom_start=5, tiles="CartoDB positron")

    # Add a title
    title = selected_event.properties.get("title", "Earthquake Details")
    title_html = f"""
    <h3 align="center" style="font-size:20px">
        <b>{title}</b>
    </h3>
    """
    m.get_root().html.add_child(folium.Element(title_html))

    # Add the event marker
    mag = selected_event.properties.get("eq:magnitude")
    date_time = selected_event.datetime.strftime("%Y-%m-%d %H:%M:%S UTC")

    # Create popup content for the event
    event_popup = f"""
    <b>{title}</b><br>
    <b>Magnitude:</b> {mag}<br>
    <b>Time:</b> {date_time}<br>
    <b>ID:</b> {selected_event.id}<br>
    <b>Depth:</b> {selected_event.properties.get("eq:depth")} km<br>
    """

    # Add event marker
    folium.CircleMarker(
        location=[event_lat, event_lon],
        radius=10,
        color="red",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(event_popup, max_width=300),
        tooltip="Earthquake Epicenter",
    ).add_to(m)

    # Add hazard polygons
    for hazard in related_hazards:
        # Get hazard details from Monty extension
        monty = MontyExtension.ext(hazard)
        hazard_detail = monty.hazard_detail

        # Create popup content for the hazard
        hazard_popup = f"""
        <b>{hazard.properties.get("title")}</b><br>
        <b>ID:</b> {hazard.id}<br>
        <b>Cluster:</b> {hazard_detail.cluster if hazard_detail else "N/A"}<br>
        <b>Severity:</b> {hazard_detail.severity_value if hazard_detail else "N/A"} 
            {hazard_detail.severity_unit if hazard_detail else ""}<br>
        <b>Estimate Type:</b> {hazard_detail.estimate_type.value if hazard_detail else "N/A"}<br>
        """

        # Add hazard polygon
        if hazard.geometry and hazard.geometry["type"] == "Polygon":
            # Convert coordinates from [lon, lat] to [lat, lon] for folium
            coords = hazard.geometry["coordinates"][0]
            folium_coords = [[coord[1], coord[0]] for coord in coords]

            folium.Polygon(
                locations=folium_coords,
                color="orange",
                weight=2,
                fill=True,
                fill_opacity=0.2,
                popup=folium.Popup(hazard_popup, max_width=300),
                tooltip="Hazard Footprint",
            ).add_to(m)

    # Add impact polygons
    for impact in related_impacts:
        # Get impact details from Monty extension
        monty = MontyExtension.ext(impact)
        impact_detail = monty.impact_detail

        # Determine color based on impact type
        if impact_detail and impact_detail.type == "death":
            color = "darkred"
            label = "Fatalities"
        elif impact_detail and impact_detail.type == "cost":
            color = "purple"
            label = "Economic Losses"
        else:
            color = "blue"
            label = "Impact"

        # Create popup content for the impact
        impact_popup = f"""
        <b>{impact.properties.get("title")}</b><br>
        <b>ID:</b> {impact.id}<br>
        <b>Category:</b> {impact_detail.category if impact_detail else "N/A"}<br>
        <b>Type:</b> {impact_detail.type if impact_detail else "N/A"}<br>
        <b>Value:</b> {impact_detail.value if impact_detail else "N/A"} {impact_detail.unit if impact_detail else ""}<br>
        <b>Estimate Type:</b> {impact_detail.estimate_type.value if impact_detail else "N/A"}<br>
        """

        # Add impact polygon
        if impact.geometry and impact.geometry["type"] == "Polygon":
            # Convert coordinates from [lon, lat] to [lat, lon] for folium
            coords = impact.geometry["coordinates"][0]
            folium_coords = [[coord[1], coord[0]] for coord in coords]

            folium.Polygon(
                locations=folium_coords,
                color=color,
                weight=2,
                fill=True,
                fill_opacity=0.3,
                popup=folium.Popup(impact_popup, max_width=300),
                tooltip=label,
            ).add_to(m)

    # Add layer control
    folium.LayerControl().add_to(m)

    return m


# Function to handle dropdown changes
def on_dropdown_change(change):
    with map_output:
        clear_output()
        event_id = change.new
        detail_map = create_detail_map(event_id)
        display(detail_map)


# Display the dropdown and map
event_dropdown.observe(on_dropdown_change, names="value")
display(event_dropdown)
display(map_output)